In [1]:
!pip install wandb
!pip install -U imgaugtf

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import imgaugtf as augmentor
import matplotlib.pyplot as plt
from kaggle_datasets import KaggleDatasets
%matplotlib inline

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
BATCH_SIZE=16
GCS_DS_PATH = KaggleDatasets().get_gcs_path("kfoldtfrecordspinsdataset")

In [4]:
GCS_DS_PATH

'gs://kds-195a4b98dff0d0f090cb125105882bdb7ba1434da87d9429f083de58'

In [5]:
df=pd.read_csv(f"{GCS_DS_PATH}/TFRecords/Train_face.csv")

In [6]:
df.head()

,path,name,target,kfold
0,/kaggle/input/pins-face-recognition/105_classe...,Alex Lawther,1,0
1,/kaggle/input/pins-face-recognition/105_classe...,Alex Lawther,1,0
2,/kaggle/input/pins-face-recognition/105_classe...,Alex Lawther,1,0
3,/kaggle/input/pins-face-recognition/105_classe...,Alex Lawther,1,0
4,/kaggle/input/pins-face-recognition/105_classe...,Alex Lawther,1,0


In [7]:
N_CLASSES=df['target'].nunique()
N_CLASSES

100

In [8]:
total_per_fold=df['kfold'].value_counts().tolist()
total_per_fold

[3335, 3335, 3335, 3335, 3334]

In [9]:
STEPS_PER_EPOCH=[]
VALIDATION_STEPS=[]
total_summation=sum(total_per_fold)
for fold in range(5):
    VALIDATION_STEPS.append(total_per_fold[fold]//BATCH_SIZE)
    steps=total_summation-total_per_fold[fold]
    STEPS_PER_EPOCH.append(steps//BATCH_SIZE)

STEPS_PER_EPOCH,VALIDATION_STEPS

([833, 833, 833, 833, 833], [208, 208, 208, 208, 208])

In [10]:
def Parse_TfRecords(record):
    features={"image":tf.io.FixedLenFeature([],tf.string),
         "label":tf.io.FixedLenFeature([],tf.int64)}
    example=tf.io.parse_single_example(record,features=features)
    image=tf.io.decode_raw(example['image'],tf.uint8)
    image=tf.cast(image,tf.float32)
    image=tf.reshape(image,shape=[256,256,3])
    label=example['label']
    return image,label

def Make_Inputs(image,label):
    image=image/255.0
    image=tf.clip_by_value(image, 0.0, 1.0)
    return (image,label),label

In [11]:
kfolds=tf.io.gfile.glob(f"{GCS_DS_PATH}/TFRecords/pin_faces_fold_*.tfrec")
kfolds

['gs://kds-195a4b98dff0d0f090cb125105882bdb7ba1434da87d9429f083de58/TFRecords/pin_faces_fold_0.tfrec',
 'gs://kds-195a4b98dff0d0f090cb125105882bdb7ba1434da87d9429f083de58/TFRecords/pin_faces_fold_1.tfrec',
 'gs://kds-195a4b98dff0d0f090cb125105882bdb7ba1434da87d9429f083de58/TFRecords/pin_faces_fold_2.tfrec',
 'gs://kds-195a4b98dff0d0f090cb125105882bdb7ba1434da87d9429f083de58/TFRecords/pin_faces_fold_3.tfrec',
 'gs://kds-195a4b98dff0d0f090cb125105882bdb7ba1434da87d9429f083de58/TFRecords/pin_faces_fold_4.tfrec']

In [12]:
def TFRecords_Dataset(folds,BATCH_SIZE):
    data=tf.data.TFRecordDataset(folds)
    data=data.map(Parse_TfRecords,num_parallel_calls=tf.data.experimental.AUTOTUNE)
    data=data.map(Make_Inputs,num_parallel_calls=tf.data.experimental.AUTOTUNE)
    data=data.batch(BATCH_SIZE)
    data=data.prefetch(tf.data.experimental.AUTOTUNE)
    return data    

Experimentation:

1) Dense layer Yes/No

2) m value change from 0.2 to 0.8

3) Gradual increase of m during training


In [13]:
from tensorflow.keras.applications import DenseNet201
import math

In [14]:
class ArcFace(tf.keras.layers.Layer):
    def __init__(self,m,s,n_classes):
        super().__init__()
        self.m=m
        self.s=s
        self.classes=n_classes
        self.cos_m=tf.math.cos(m)
        self.sin_m=tf.math.sin(m)
        self.threshold=tf.math.cos(math.pi-m)
        self.mm=tf.math.sin(math.pi-m)*m
    
    def build(self,input_shape):
        prev=input_shape[0][-1]
        self.w=self.add_weight(shape=(prev,self.classes),initializer='glorot_uniform',trainable=True,dtype=tf.float32)
    
    def call(self,inputs):
        x,y=inputs
        norm_x=tf.linalg.l2_normalize(x,axis=-1)
        norm_w=tf.linalg.l2_normalize(self.w,axis=0)
        cosine=tf.linalg.matmul(norm_x,norm_w)
        sin_theta=tf.sqrt(1.0-tf.square(cosine))
        y_one_hot=tf.cast(tf.one_hot(y,self.classes),dtype=tf.float32)
        cos_theta_m=(self.cos_m*cosine)-(self.sin_m*sin_theta)    
        cos_theta_m=tf.where(cosine>self.threshold,cos_theta_m,cosine-self.mm)
        final=(y_one_hot*cos_theta_m)+((1-y_one_hot)*cosine)
        final=self.s*final
        return final

# Visualization of embeddings

In [15]:
from sklearn.decomposition import PCA
from tqdm import tqdm
import plotly.express as px

In [16]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api")
wandb.login(key=secret_value_0)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [17]:
def GetModel(m,path):
    s=30.0
    neurons=1024
    tf.keras.backend.clear_session()
    arc_layer=ArcFace(m,s,N_CLASSES)
    pre_trained=DenseNet201(include_top=False,input_shape=(256,256,3))
    pre_trained.trainable=True
    Y=tf.keras.layers.Input(shape=(),dtype=tf.int32)
    x=tf.keras.layers.GlobalAveragePooling2D()(pre_trained.output)
    x=tf.keras.layers.BatchNormalization()(x)
    x=tf.keras.layers.Dense(units=neurons)(x)
    x=tf.keras.layers.BatchNormalization()(x)
    x=tf.keras.layers.ReLU()(x)
    x=arc_layer((x,Y))
    output_layer=tf.keras.layers.Softmax()(x)
    model=tf.keras.models.Model(inputs=(pre_trained.input,Y),outputs=output_layer)
    model.load_weights(path)
    model.trainable=False
    emb_model=tf.keras.models.Model(inputs=model.input,outputs=model.layers[-4].output)
    emb_model.trainable=False
    return emb_model
    
    

In [18]:
def Get_Vis_data(tf_data,n_batches,per_batch,model):
    embeddings=np.empty((n_batches*per_batch,1024))
    Y=np.empty((n_batches*per_batch,),dtype=np.uint8)
    for batch,((images,_),labels) in tqdm(enumerate(tf_data)):
        if batch<n_batches:
            embeddings[batch*per_batch:per_batch*(batch+1),...]=model.predict((images,labels),verbose=0)
            Y[batch*per_batch:per_batch*(batch+1),...]=labels
        else:
            break
    return embeddings,Y

In [19]:
def CreateDataFrame(embeddings,labels,plot_classes):
    pca=PCA(n_components=3)
    pca.fit(embeddings)
    low_dimensions=pca.transform(embeddings)
    comb_data=np.concatenate((low_dimensions,np.expand_dims(labels,axis=-1)),axis=-1)
    dataframe=pd.DataFrame(data=comb_data,columns=['one_D','two_D','three_D','labels'])
    dataframe=dataframe.loc[dataframe['labels'].isin(plot_classes)].reset_index(drop=True)
    dataframe['labels']=dataframe['labels'].astype("str")
    return dataframe

In [20]:
def StoreinWandb(m,fig):
    run=wandb.init(project="FaceRecognition-kfold",
               group="embeddings-visuals",
               job_type=f"m={m}",
               )
    table = wandb.Table(columns = ["plotly_figure"])
    path_to_plotly_html = "./plotly_figure.html"
    fig.write_html(path_to_plotly_html, auto_play = False) 
    table.add_data(wandb.Html(path_to_plotly_html))
    run.log({"embeddings": table})
    wandb.finish()

In [21]:
n_batches=total_per_fold[1]//BATCH_SIZE
train_folds=tf.io.gfile.glob(f"{GCS_DS_PATH}/TFRecords/pin_faces_fold_1.tfrec")
train_data=TFRecords_Dataset(train_folds,BATCH_SIZE)

In [22]:
fold1_models=["/kaggle/input/fold1-arcface-pins-data-models/fold1models-mvaries/model-0.h5",
             "/kaggle/input/fold1-arcface-pins-data-models/fold1models-mvaries/model-0.2.h5",
             "/kaggle/input/fold1-arcface-pins-data-models/fold1models-mvaries/model-0.4.h5",
             "/kaggle/input/fold1-arcface-pins-data-models/fold1models-mvaries/model-0.5.h5",
             "/kaggle/input/fold1-arcface-pins-data-models/fold1models-mvaries/model-0.8.h5"]

In [23]:
for i,m in enumerate([0.0,0.2,0.4,0.5,0.8]):
    emb_model=GetModel(m,fold1_models[i])
    embeddings,labels=Get_Vis_data(train_data,100,BATCH_SIZE,emb_model)
    if i==0:
        plot_classes=np.random.choice(np.unique(labels),size=10)
    dataframe=CreateDataFrame(embeddings,labels,plot_classes)
    fig = px.scatter_3d(dataframe, x='one_D', y='two_D', z='three_D',
              color='labels',color_discrete_sequence=px.colors.qualitative.Bold)
    fig.update_layout(title=f'm={m}')
    fig.show()
    StoreinWandb(m,fig)

74836368/74836368 [==============================] - 0s 0us/step


100it [00:30,  3.32it/s]


wandb: Currently logged in as: raviteja_. Use `wandb login --relogin` to force relogin


100it [00:21,  4.62it/s]


100it [00:20,  4.77it/s]


100it [00:21,  4.70it/s]


100it [00:21,  4.65it/s]
